## Prepare Muse

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
m_path = r'\\ir1-nasrst-p03\nasuni\Bru\static\lab_data\Nyx\OvernightsFOS'
output_dir = r'\\ir1-nasrst-p03\nasuni\Bru\static\lab_data\Nyx\OvernightsFOS_proc'

In [3]:
files=os.listdir(m_path)

In [4]:
files

['22.csv',
 'ID143_2024-02-15--22-10-35_763601666229345239.csv',
 'ID159_2024-02-16--00-08-16_656872965849429911.csv',
 'ID160_2024-02-13--23-45-53_1315356916044442319.csv',
 'ID166_2024-02-14--21-58-07_2287995164915597982.csv',
 'ID166_2024-02-21--23-34-37_1099229584771097297.csv',
 'ID170_2024-02-14--21-26-44_7233461900097302942.csv',
 'ID171_2024-02-20--00-00-01_736228705208675944.csv']

In [5]:
from scipy.signal import butter, sosfilt
def filter_down(eeg_data, lowcut=.05, highcut=25, order=50, sample_in=100, sample_out=32):

    # Calculate the Nyquist frequency
    nyquist_freq = 0.5 * sample_in

    # Calculate the filter coefficients
    sos = butter(order, [lowcut / nyquist_freq, highcut / nyquist_freq], btype='bandpass', output='sos')

    # Apply the filter to the EEG data
    filtered_eeg = sosfilt(sos, eeg_data)

    #fandd=fast_resample(filtered_eeg, fs_out=sample_in, fs_in=sample_out)
    return filtered_eeg[::2]#fandd

In [20]:
for i, file in enumerate(files):    
    EEG = pd.read_csv(os.path.join(m_path,file), usecols=['RAW_AF7'])
    f_d_eeg=filter_down(EEG, lowcut=.05, highcut=62, order=5, sample_in=256, sample_out=125)
    n_epochs=int(f_d_eeg.shape[0]//(30*100))
    x = np.asarray(np.split(f_d_eeg[:n_epochs*(30*100)], n_epochs)).astype(np.float32)[:, :, 0]
    # Save
    filename = str(i) + '.npz'
    save_dict = {
        "x": x, 
        "y": 'NA', 
        "fs": 125,
        "ch_label": 'AF7',
        "header_raw": 'NA',
        "header_annotation": 'NA',
    }
    np.savez(os.path.join(output_dir, filename), **save_dict)


In [21]:
f=np.load(os.path.join(output_dir, filename))

In [22]:
f['x'].shape

(944, 3000)

In [ ]:
# Save
filename = str(i) + '.npz'
save_dict = {
    "x": f_d_eeg, 
    "y": 'NA', 
    "fs": 125,
    "ch_label": 'AF7',
    "header_raw": 'NA',
    "header_annotation": 'NA',
}
np.savez(os.path.join(args.output_dir, filename), **save_dict)